### Combine Title and Text


In [1]:
import os
import numpy as np
os.chdir('C:\\Users\\lgu\\Downloads\\pizza_request_dataset.tar\\pizza_request_dataset\\pizza_request_dataset' )

In [2]:
#Load the data

import pandas, json
from sklearn.feature_extraction.text import *
import re
import numpy as np


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report


json_data = json.load(open('pizza_request_dataset.json'))
df=pandas.io.json.json_normalize(json_data)
print(df.columns.values)
print(df.shape)

['giver_username_if_known' 'in_test_set'
 'number_of_downvotes_of_request_at_retrieval'
 'number_of_upvotes_of_request_at_retrieval' 'post_was_edited' 'request_id'
 'request_number_of_comments_at_retrieval' 'request_text'
 'request_text_edit_aware' 'request_title'
 'requester_account_age_in_days_at_request'
 'requester_account_age_in_days_at_retrieval'
 'requester_days_since_first_post_on_raop_at_request'
 'requester_days_since_first_post_on_raop_at_retrieval'
 'requester_number_of_comments_at_request'
 'requester_number_of_comments_at_retrieval'
 'requester_number_of_comments_in_raop_at_request'
 'requester_number_of_comments_in_raop_at_retrieval'
 'requester_number_of_posts_at_request'
 'requester_number_of_posts_at_retrieval'
 'requester_number_of_posts_on_raop_at_request'
 'requester_number_of_posts_on_raop_at_retrieval'
 'requester_number_of_subreddits_at_request' 'requester_received_pizza'
 'requester_subreddits_at_request'
 'requester_upvotes_minus_downvotes_at_request'
 'reques

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(0)

from sklearn.cross_validation import train_test_split

train, dev = train_test_split(df, test_size = 0.3)

train_labels=train["requester_received_pizza"]
train_data = train
del train_data["requester_received_pizza"]

dev_labels=dev["requester_received_pizza"]
dev_data = dev
del dev_data["requester_received_pizza"]

print ('training label shape:', train_labels.shape)
print ('test shape:', train_data.shape)
print ('test label shape:', dev_labels.shape)
print ('test shape:', dev_data.shape)

training label shape: (3969,)
test shape: (3969, 32)
test label shape: (1702,)
test shape: (1702, 32)


In [8]:
train_text=train_data['request_text'].as_matrix()
train_title=train_data['request_title'].as_matrix()
train_all = train_text+train_title

dev_text=dev_data['request_text'].as_matrix()
dev_title=dev_data['request_title'].as_matrix()
dev_all = dev_text+dev_title

###baseline: no reprocess, no feature selection

In [9]:

def empty_preprocessor(s):
    return s


vectorizer_process = CountVectorizer(preprocessor =empty_preprocessor,analyzer='word',stop_words='english' )
word_matrix_process= vectorizer_process.fit_transform(train_all)
dev_matrix_process = vectorizer_process.transform(dev_all)


model_LG = LogisticRegression(penalty ='l2',C=12)
model_LG.fit(word_matrix_process, train_labels)


f1_score=metrics.f1_score(dev_labels,model_LG.predict(dev_matrix_process),average='binary')
print(f1_score)
print(metrics.precision_score(dev_labels,model_LG.predict(dev_matrix_process)))
print(metrics.recall_score(dev_labels,model_LG.predict(dev_matrix_process)))

0.319123020706
0.334183673469
0.305361305361


###Add preprocess and L1 feature selection 

make sure none of the doc is empty

In [23]:
from scipy import sparse
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix, hstack
# total_sum=csr_matrix.sum(word_matrix_process,axis=1)
# total_sum.shape
np.where(total_sum==0 )[0]
# total_sum[1:5]
# np.zeros(total_sum[:,0])

matrix([], shape=(1, 0), dtype=int64)

In [57]:
def first_preprocessor(s):
    #convert to lowercase
    s=s.lower()
    s=re.sub("[,.!?:;/~*]"," ",s)
#     #remove duplicated 0s and 1s
    s=re.sub("[0-9]*","",s)
#     #remove number longer than 5 digit
    s=re.sub("[0-9]{5,}","",s)
#     #remove word longer than 20 digit
#     s=re.sub("[0-9a-z]{20,}","",s)
#     #remove stem end with 'ly'
    s=re.sub("ly\s"," ",s)
#     s=re.sub("ly\Z"," ",s)
#     #remove plural form
    s=re.sub("s\s"," ",s)
    s=re.sub("s\Z"," ",s)
#     #remove _ as the end of word
    s=re.sub("[_]+"," ",s)
#     #remove _ as start of the word
    s=re.sub("\s[_]+"," ",s)
#     #remove stem end with 'ful'
#     s=re.sub("ful\s"," ",s)
#     s=re.sub("ful\Z"," ",s)
#     #remove stem end with 'able'
#     s=re.sub("able\s"," ",s)
#     s=re.sub("able\Z"," ",s)
#     #remove stem end with 'ness'
    s=re.sub("ness\s"," ",s)
    s=re.sub("ing\s"," ",s)
#     s=re.sub("ing\Z"," ",s)
#     #remove stop words
#     s=re.sub(" about "," ",s)
    #remove words that are too short
    s=re.sub("\s[0-9a-z]{1,2}\s"," ",s)
    s=re.sub("\s[0-9a-z]{1,2}\Z"," ",s)

    return s
def empty_preprocessor(s):
    return s

# train_data_array=train_data['request_text'].as_matrix()
vectorizer_process = CountVectorizer(preprocessor =empty_preprocessor,analyzer='word',stop_words='english' )
word_matrix_process= vectorizer_process.fit_transform(train_all)
dev_matrix_process = vectorizer_process.transform(dev_all)

def tune_para(L1,L2):
    model_LG_L1 = LogisticRegression(penalty ='l1',C=570)
    model_LG_L1.fit(word_matrix_process, train_labels)

    index=[]
    for i,z in enumerate(np.sum(np.abs(model_LG_L1.coef_),axis=0) ):
        if z!=0:
            index.append(i)
#     print(word_matrix_process.shape)
#     print(len(index))
    # print('When C= %f, the number of learned weights that are not equal to zero is %d' %(c,len(index)))
    #train the logistic regression with the features after pruning

    model_LG_L2 = LogisticRegression(penalty ='l2',C=12 )
    model_LG_L2.fit(word_matrix_process[:,index], train_labels)

    # accuracy.append(metrics.accuracy_score(test_labels,model_LG_p6_l2_retrain.predict(vectorizer_p6.transform(test_data)[:,index])))

    f1_score=metrics.f1_score(dev_labels,model_LG_L2.predict(dev_matrix_process[:,index]),average='binary')
#     print(f1_score)
#     print(metrics.precision_score(dev_labels,model_LG_L2.predict(dev_matrix_process[:,index])))
# print(metrics.recall_score(dev_labels,model_LG.predict(dev_matrix_process)))
    return f1_score

Loop in parameters and get the best parameters for C of L1 and L2

In [194]:
np.random.seed(0)
max=0
l1=0
l2=0
for i in range(500,700,10):
    for j in range(1,50,1):
        acc=tune_para(i,j)
        if acc>max:
            max=acc
            l1=i
            l2=j
        print('When l1=%i ,l2=%i , taccuracy is %f' %(i,j,acc))

 

When l1=500 ,l2=1 , taccuracy is 0.343529
When l1=500 ,l2=2 , taccuracy is 0.341578
When l1=500 ,l2=3 , taccuracy is 0.340828
When l1=500 ,l2=4 , taccuracy is 0.332547
When l1=500 ,l2=5 , taccuracy is 0.337264
When l1=500 ,l2=6 , taccuracy is 0.341578
When l1=500 ,l2=7 , taccuracy is 0.342169
When l1=500 ,l2=8 , taccuracy is 0.341232
When l1=500 ,l2=9 , taccuracy is 0.334123
When l1=500 ,l2=10 , taccuracy is 0.333728
When l1=500 ,l2=11 , taccuracy is 0.338863
When l1=500 ,l2=12 , taccuracy is 0.343934
When l1=500 ,l2=13 , taccuracy is 0.331375
When l1=500 ,l2=14 , taccuracy is 0.334906
When l1=500 ,l2=15 , taccuracy is 0.336867
When l1=500 ,l2=16 , taccuracy is 0.336516
When l1=500 ,l2=17 , taccuracy is 0.344009
When l1=500 ,l2=18 , taccuracy is 0.332147
When l1=500 ,l2=19 , taccuracy is 0.333333
When l1=500 ,l2=20 , taccuracy is 0.343195
When l1=500 ,l2=21 , taccuracy is 0.352526
When l1=500 ,l2=22 , taccuracy is 0.334112
When l1=500 ,l2=23 , taccuracy is 0.336114
When l1=500 ,l2=24 ,

KeyboardInterrupt: 

When l1=550 ,l2=45 , accuracy is 0.355556 empty_preprocessor

In [63]:
model_LG_L1 = LogisticRegression(penalty ='l1',C=550)
model_LG_L1.fit(word_matrix_process, train_labels)

index=[]
for i,z in enumerate(np.sum(np.abs(model_LG_L1.coef_),axis=0) ):
    if z!=0:
        index.append(i)
print(len(index))

2621


###PCA

In [69]:
from sklearn.decomposition import PCA
i=1000
pca_mod = PCA(n_components = i)
word_matrix_process_pca=pca_mod.fit_transform(word_matrix_process.toarray()[:,index])
print('For the first %i element, %f of the total variance in the training data is explained ' %(i,sum(pca_mod.explained_variance_ratio_)) )


For the first 1000 element, 0.961224 of the total variance in the training data is explained 


In [73]:
dev_matrix_process_pca=pca_mod.transform(dev_matrix_process.toarray()[:,index])
# dev_matrix_process_pca.shape

model_LG_L2 = LogisticRegression(penalty ='l2',C=45 )
model_LG_L2.fit(word_matrix_process_pca, train_labels)
f1_score=metrics.f1_score(dev_labels,model_LG_L2.predict(dev_matrix_process_pca),average='binary')
print(f1_score)
print(metrics.precision_score(dev_labels,model_LG_L2.predict(dev_matrix_process_pca)))
print(metrics.recall_score(dev_labels,model_LG_L2.predict(dev_matrix_process_pca)))
print(metrics.accuracy_score(dev_labels,model_LG_L2.predict(dev_matrix_process_pca)))

0.330322580645
0.369942196532
0.298368298368
0.695064629847


###Narrative


In [82]:
def get_featureCount(feature_list,matrix,vectorizer):
    a=csr_matrix((matrix.shape[0],1), dtype=np.int8)
    for word in feature_list:
        print(word)
        if vectorizer.vocabulary_.get(word)!=None:
            b=matrix[:,vectorizer_process.vocabulary_.get(word)]
            a= hstack([a,b],format= 'csr')
    return a


In [147]:
vectorizer_process = CountVectorizer(preprocessor =empty_preprocessor,analyzer='word',stop_words='english' )
word_matrix_process= vectorizer_process.fit_transform(train_all)
dev_matrix_process = vectorizer_process.transform(dev_all)

desire = ['party','birthday','boyfriend','girlfriend','date','drinks','drunk','wasted','invite','invited','celebrate',
          'celebrating','game','games','movie','beer','crave','craving']

narrative_count_abs=get_featureCount(desire,word_matrix_process,vectorizer_process) 
narrative_count_adj=csr_matrix.sum(narrative_count_abs,axis=1)/(csr_matrix.sum(word_matrix_process,axis=1) + 0.0)*100
print(np.max(narrative_count_adj))
print(narrative_count_adj.shape)
# print(type(narrative_count_adj.toarray()))
np.percentile( narrative_count_adj[:,0] , np.arange(0, 100, 10))
# a=csr_matrix((2,1), dtype=np.int8)
# print(a)

21.4285714286
(3969, 1)


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  2.94117647])

In [181]:
print(narrative_count_adj.shape)
print(narrative_count_abs.toarray()[0])
train_all[0]
# np.median(narrative_count_adj)
a=narrative_count_adj 
print(  np.nonzero(a)[0].shape)

(3969, 1)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
(1, 734)


In [139]:
df = pandas.DataFrame([1,7,8,2,3,4,5,6,6,6,7], columns=['investment'])
print(df)
df['decile'] = pandas.qcut(df['investment'], 3, labels=False)
print(df)


    investment
0            1
1            7
2            8
3            2
4            3
5            4
6            5
7            6
8            6
9            6
10           7
    investment  decile
0            1       0
1            7       2
2            8       2
3            2       0
4            3       0
5            4       0
6            5       1
7            6       1
8            6       1
9            6       1
10           7       2


In [193]:
narrative_count_adj[np.nonzero(narrative_count_adj)].as_matrix()

# list_count=pandas.DataFrame(narrative_count_adj[np.nonzero(narrative_count_adj)],columns=['desire'])
# # # print(desire)
# np.nonzero(list_count['desire'])[0]
# list_count[:np.nonzero(list_count['desire'])[0]]
# list_count['decile'] = pandas.qcut(list_count[np.nonzero(list_count['desire'])], 10, labels=False)
# print(sum(list_count['decile']==1))

AttributeError: 'matrix' object has no attribute 'as_matrix'

In [199]:
import nltk
import string
import os

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
path = '/opt/datacourse/data/parts'
token_dict = {}
stemmer = SnowballStemmer("english", ignore_stopwords=True)

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer_process = CountVectorizer(analyzer='word',stop_words='english',preprocessor = empty_preprocessor,tokenizer=tokenize)
word_matrix_process= vectorizer_process.fit_transform(train_all)
dev_matrix_process = vectorizer_process.transform(dev_all)

model_LG_L1 = LogisticRegression(penalty ='l1',C=560)
model_LG_L1.fit(word_matrix_process, train_labels)

index=[]
for i,z in enumerate(np.sum(np.abs(model_LG_L1.coef_),axis=0) ):
    if z!=0:
        index.append(i)
            
model_LG = LogisticRegression(penalty ='l2',C=19)#C from the above test
model_LG.fit(word_matrix_process[:,index], train_labels)

print('f1_score: %f' %metrics.f1_score(dev_labels,model_LG.predict(dev_matrix_process[:,index]),average='binary'))
print('accuracy_score: %f' %metrics.precision_score(dev_labels,model_LG.predict(dev_matrix_process[:,index])))
print('recall_score: %f' %metrics.recall_score(dev_labels,model_LG.predict(dev_matrix_process[:,index])))


f1_score: 0.332564
accuracy_score: 0.329519
recall_score: 0.335664
